![](https://static.wixstatic.com/media/4205be_693bbdf8070b461186014060ec420cc7~mv2.jpg/v1/fill/w_568,h_386,al_c,q_80,usm_0.66_1.00_0.01,enc_avif,quality_auto/4205be_693bbdf8070b461186014060ec420cc7~mv2.jpg)

# 1. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import cv2
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from tqdm import tqdm

# Magic command for displaying plots in Jupyter notebooks
%matplotlib inline

# 6. Building the CNN Model

In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()

        # --- CÁC LỚP TÍCH CHẬP GIỮ NGUYÊN ---
        # Block 1
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)

        # Block 2
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)

        # Block 3
        self.conv3 = nn.Conv2d(128, 512, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)

        # Block 4
        self.conv4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout4 = nn.Dropout(0.25)

        # --- CÁC LỚP KẾT NỐI ĐẦY ĐỦ (DENSE LAYERS) ---

        # <<< THAY ĐỔI QUAN TRỌNG Ở ĐÂY >>>
        # Kích thước đầu vào của lớp fc1 được cập nhật cho ảnh 112x112
        # Kích thước sau 4 lớp pooling: 112 -> 56 -> 28 -> 14 -> 7.
        # Kích thước vector phẳng = 512 (kênh) * 7 * 7 = 25088
        self.fc1 = nn.Linear(512 * 7 * 7, 256)

        self.bn5 = nn.BatchNorm1d(256)
        self.relu5 = nn.ReLU()
        self.dropout5 = nn.Dropout(0.25)

        self.fc2 = nn.Linear(256, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.relu6 = nn.ReLU()
        self.dropout6 = nn.Dropout(0.25)

        # Output layer
        self.fc3 = nn.Linear(512, num_classes)


    def forward(self, x):
        # --- LUỒNG DỮ LIỆU ĐI TIẾP VẪN GIỮ NGUYÊN ---
        # Conv block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        # Conv block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        # Conv block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.dropout3(x)

        # Conv block 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        x = self.dropout4(x)

        # Flatten
        x = torch.flatten(x, 1)

        # Dense layers
        x = self.fc1(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.dropout5(x)

        x = self.fc2(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.dropout6(x)

        # Output layer
        x = self.fc3(x)

        return x

# --- CÁC PHẦN KHỞI TẠO KHÁC GIỮ NGUYÊN ---
# Khởi tạo mô hình
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionCNN(num_classes=7).to(device)

# Định nghĩa optimizer và loss function
# Thêm L2 regularization thông qua weight_decay
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# In ra cấu trúc model để kiểm tra
print(model)

EmotionCNN(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.25, inplace=False)
  (conv3): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout3): Dropout(p=0.25, inplace=False)
  (conv4): Conv2d(512, 512, kernel_size

# 9. Evaluating the Model


In [ ]:
# 2. Cấu hình
MODEL_PATH = 'best_model.pth' # <<< Sửa lại tên file .pth cho đúng
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 3. Tải model
model = EmotionCNN().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# 4. Định nghĩa transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 5. Khởi tạo webcam và bộ dò tìm khuôn mặt
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# <<< THÊM MỚI: Biến để kiểm soát việc lật camera >>>
flip_frame = False

# --- VÒNG LẶP CHÍNH ---
while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc frame từ webcam.")
        break

    # <<< CẢI TIẾN: Lật camera nếu biến flip_frame là True >>>
    if flip_frame:
        frame = cv2.flip(frame, 1) # Lật theo trục dọc

    # Xử lý phím bấm
    key = cv2.waitKey(1) & 0xFF

    # Nhấn 'q' để thoát
    if key == ord('q'):
        break

    # <<< CẢI TIẾN: Nhấn 'f' để bật/tắt chế độ lật camera >>>
    if key == ord('f'):
        flip_frame = not flip_frame

    # Chuyển sang ảnh xám để dò tìm
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Xử lý từng khuôn mặt tìm được
    for (x, y, w, h) in faces:
        # Vẽ khung chữ nhật quanh khuôn mặt
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Cắt riêng vùng khuôn mặt
        face_roi = frame[y:y+h, x:x+w]

        # Chuyển đổi và áp dụng transform
        pil_image = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        image_tensor = transform(pil_image).unsqueeze(0).to(DEVICE)

        # Dự đoán cảm xúc
        with torch.no_grad():
            output = model(image_tensor)

            # <<< CẢI TIẾN: Sử dụng Softmax để lấy xác suất phần trăm >>>
            probabilities = torch.nn.functional.softmax(output, dim=1)[0] # Lấy tensor xác suất

            # Lấy cảm xúc có xác suất cao nhất để hiển thị chính
            confidence, predicted_idx = torch.max(probabilities, 0)
            main_emotion = EMOTION_LABELS[predicted_idx.item()]
            main_confidence = confidence.item()

        # Hiển thị cảm xúc chính lên trên khung chữ nhật
        main_text = f"{main_emotion} ({main_confidence:.2f})"
        cv2.putText(frame, main_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # <<< CẢI TIẾN: Hiển thị tất cả các phần trăm ở góc màn hình >>>
        # Tạo một background mờ để dễ đọc chữ
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (250, 220), (0, 0, 0), -1)
        alpha = 0.6  # Độ trong suốt
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        # In từng dòng cảm xúc và phần trăm
        for i, (label, prob) in enumerate(zip(EMOTION_LABELS, probabilities)):
            text = f"{label}: {prob.item():.2%}"
            text_y = 30 + i * 28 # Vị trí y cho mỗi dòng

            # Highlight cảm xúc có % cao nhất
            text_color = (0, 255, 0) if label == main_emotion else (255, 255, 255)

            cv2.putText(frame, text, (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)


    # Hiển thị khung hình
    cv2.imshow('Emotion Detection', frame)

# Dọn dẹp
cap.release()
cv2.destroyAllWindows()